In [1]:
import torch
import torchvision
from torchvision import transforms,utils,datasets,models
from torch.autograd import Variable
import torch.nn as nn
import shutil
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from PIL import Image, ImageFilter
import time
start=time.time()
infile="/home/user01/envoriments/data/validate/"
outfile="/home/user01/envoriments/data/validate_error/"
use_gpu=torch.cuda.is_available()
small_class_num={ 'coat':8, 'collar':5, 'lapel':5, 'neck':5, 'neckline':10, 'pant':6, 'sleeve':9, 'skirt':6}
retrain_model_class={ 'coat':'densenet_fashionai_coat_en.pkl',
                     'collar':'densenet_fashionai_collar_en.pkl',
                     'lapel':'densenet_fashionai_lapel.pkl',
                     'neck':'densenet_fashionai_neck.pkl', 
                     'neckline':'densenet_fashionai_neckline_en.pkl', 
                     'pant':'densenet_fashionai_pant.pkl',
                     'sleeve':'densenet_fashionai_sleeve_en.pkl',
                     'skirt':'densenet_fashionai_skirt.pkl'}
images_class={ 'coat':'coat_length_labels',
                     'collar':'collar_design_labels',
                     'lapel':'lapel_design_labels',
                     'neck':'neck_design_labels', 
                     'neckline':'neckline_design_labels', 
                     'pant':'pant_length_labels',
                     'sleeve':'sleeve_length_labels',
                     'skirt':'skirt_length_labels'}

def image_transfroms():# 把数据转换成模型指定大小，并转换成tensor类型
    return transforms.Compose([
        transforms.Scale(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
def getdataset(phase):
    imagesfile=images_class[phase]
    validate_dataset=torchvision.datasets.ImageFolder("/home/user01/envoriments/data/validate/"+imagesfile,
                                             transform=transforms.Compose([
                                                transforms.Scale(224),
                                                transforms.CenterCrop(224),
                                                transforms.ToTensor()
                                             ]))
    validate_dataloader=torch.utils.data.DataLoader(validate_dataset,batch_size=1,shuffle=True,num_workers=1)
    return validate_dataset,validate_dataloader            

def train_model(model,criterion,phase):
    dataset,dataloader=getdataset(phase)
    best_acc=0.0
    model.eval()
    running_loss=0.0
    running_corrects=0
    for data in dataloader:
        inputs,labels=data
        print(labels)
        if use_gpu:
            inputs,labels=Variable(inputs.cuda()),Variable(labels.cuda())
        else:
            inputs,labels=Variable(inputs),Variable(labels)
        outputs=model(inputs)
        _,preds=torch.max(outputs.data,1)#返回第一维的值和索引
        loss=criterion(outputs,labels)
        running_loss+=loss.data[0]#?
        running_corrects+=torch.sum(preds==labels.data)
    epoch_loss=running_loss/len(dataset)
    epoch_acc=running_corrects/len(dataset)
    print('loss:{:.4f} acc:{:.4f}'.format(epoch_loss,epoch_acc))
def move_img(phase,model):#从文件夹里读取每一张图片并送入训练好的模型
    err_num=0
    newinfile=infile+images_class[phase]
    newoutfile=outfile+images_class[phase]
    small_classes=os.listdir(newinfile)
    small_classes.sort()
   
    
    for number,small_class in enumerate(small_classes):
        images=os.listdir(newinfile+"/"+small_class)
        if not os.path.exists(os.path.join(newoutfile,small_class)):
            os.makedirs(os.path.join(newoutfile,small_class))
        for image in images:
            old_name=os.path.join(newinfile,small_class,image)
            img=Image.open(old_name)
            img_tensor=image_transfroms()(img)
            img_tensor.resize_(1,3,224,224)
            img_tensor=Variable(img_tensor)
            outputs=model(img_tensor)
            outputs=torch.nn.functional.softmax(outputs)
            _,preds=torch.max(outputs.data,1)
            
            
            if  preds.numpy()[0][0]!=number:
                err_num+=1
                new_name=os.path.join(newoutfile,small_class,image)        
                shutil.copy(old_name,new_name)
    print("移动时间为{}".format(time.time()-start))            
    print("共有{}个".format(err_num))
    
def get_result(phase):
    model_dense=models.densenet169()
    out_features=small_class_num[phase]
    model_dense.classifier=torch.nn.Linear(1664,out_features)
    url=retrain_model_class[phase]
    model_dense.load_state_dict(torch.load(url))
    
    model_dense.eval()
    criterion=torch.nn.CrossEntropyLoss()
    #model=train_model(model_dense,criterion,phase)
    move_img(phase,model_dense)
get_result('collar')
get_result('coat')
get_result('neck')
get_result('sleeve')
get_result('lapel')
get_result('neckline')
get_result('pant')
get_result('skirt')


移动时间为616.563530921936
共有102个
移动时间为1435.0048596858978
共有252个
移动时间为1789.451654434204
共有109个
移动时间为2424.509303331375
共有179个
移动时间为3394.5932869911194
共有71个


In [7]:
import torch
import torchvision
from torchvision import transforms,utils,datasets,models
from torch.autograd import Variable
import torch.nn as nn
import shutil
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from PIL import Image, ImageFilter
import time
start=time.time()
infile="/home/user01/envoriments/data/validate_error/"
outfile="/home/user01/envoriments/data/validate/"
small_class_num={ 'coat':8, 'collar':5, 'lapel':5, 'neck':5, 'neckline':10, 'pant':6, 'sleeve':9, 'skirt':6}
retrain_model_class={ 'coat':'densenet_fashionai_coat_en.pkl',
                     'collar':'densenet_fashionai_collar_en.pkl',
                     'lapel':'densenet_fashionai_lapel.pkl',
                     'neck':'densenet_fashionai_neck.pkl', 
                     'neckline':'densenet_fashionai_neckline_en.pkl', 
                     'pant':'densenet_fashionai_pant.pkl',
                     'sleeve':'densenet_fashionai_sleeve_en.pkl',
                     'skirt':'densenet_fashionai_skirt.pkl'}
images_class={ 'coat':'coat_length_labels',
                     'collar':'collar_design_labels',
                     'lapel':'lapel_design_labels',
                     'neck':'neck_design_labels', 
                     'neckline':'neckline_design_labels', 
                     'pant':'pant_length_labels',
                     'sleeve':'sleeve_length_labels',
                     'skirt':'skirt_length_labels'}

def move_img(phase):#从文件夹里读取每一张图片并送入训练好的模型
    err_num=0
    newinfile=infile+images_class[phase]
    newoutfile=outfile+images_class[phase]
    small_classes=os.listdir(newinfile)
    small_classes.sort()
    for number,small_class in enumerate(small_classes):
        images=os.listdir(newinfile+"/"+small_class)
        for image in images:
            if  image.startswith("cr") or image.startswith("ro") :
                old_name=os.path.join(newinfile,small_class,image)
                err_num+=1
                new_name=os.path.join(newoutfile,small_class,image)        
                shutil.copy(old_name,new_name)
    print("移动时间为{}".format(time.time()-start))            
    print("共有{}个".format(err_num))
    
move_img('collar')
move_img('coat')
move_img('neck')
move_img('sleeve')
move_img('lapel')
move_img('neckline')
move_img('pant')
move_img('skirt')    

移动时间为0.04490971565246582
共有300个
移动时间为0.12105131149291992
共有312个
移动时间为0.15578985214233398
共有222个
移动时间为0.19291210174560547
共有358个
移动时间为0.21249032020568848
共有204个
移动时间为0.2614712715148926
共有504个
移动时间为0.2830080986022949
共有218个
移动时间为0.29685211181640625
共有142个
